## La Liga: A Brief Analysis ##

# 1단계: 라이브러리 임포트 (Import Libraries)

In [1]:
# 1-1. 라이브러리 임포트 (Import Libraries)
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# 1-2. 데이터 로드 및 'date' 컬럼 타입 변환 (Load Data and Convert 'date' column)
df = pd.read_csv('la_liga_2014_2025_all_matches_final.csv')
df['date'] = pd.to_datetime(df['date'])

# 1-3. 정확한 시즌 라벨링 함수 정의 (Define Accurate Season Labeling Function)
# 제공해주신 시즌 정보를 바탕으로, 날짜가 어느 시즌에 속하는지 구분합니다.
season_end_dates = {
    '2014-2015': '2015-05-24', '2015-2016': '2016-05-16', '2016-2017': '2017-05-22',
    '2017-2018': '2018-05-21', '2018-2019': '2019-05-27', '2019-2020': '2020-07-20',
    '2020-2021': '2021-05-24', '2021-2022': '2022-05-23', '2022-2023': '2023-06-05',
    '2023-2024': '2024-05-27', '2024-2025': '2025-05-26'
}
# 날짜 문자열을 datetime 객체로 변환합니다.
season_end_dates_dt = {season: pd.to_datetime(end_date) for season, end_date in season_end_dates.items()}

def get_season(date):
    for season, end_date in season_end_dates_dt.items():
        if date <= end_date:
            # 해당 시즌의 시작일을 찾습니다. (이전 시즌의 종료일 + 1일)
            previous_season_year = int(season.split('-')[0]) - 1
            previous_season = f"{previous_season_year}-{previous_season_year+1}"
            start_date = season_end_dates_dt.get(previous_season, pd.to_datetime('1900-01-01')) + pd.Timedelta(days=1)
            
            if date >= start_date:
                return season
    return "Unknown"

df['season'] = df['date'].apply(get_season)

print("Data loading and season labeling complete.")

# 1-4. 시즌별 경기 수 확인 (Verify Match Count per Season)
# La Liga는 20개 팀이 홈&어웨이로 경기를 치르므로, 한 시즌은 (20 * 19) / 2 = 190경기가 아닌, 팀당 38경기씩 총 380경기가 맞습니다.
# (팀당 38경기 * 20팀) / 2 = 380 경기.
# 이 검증을 통해 데이터가 누락 없이 잘 포함되었는지 확인합니다.
match_counts = df.groupby('season').size().reset_index(name='match_count')
print("\n--- Season by Match Count ---")
print(match_counts)

# 시각화를 통해 한눈에 확인
fig_match_counts = px.bar(match_counts, x='season', y='match_count', text_auto=True,
                          title='Number of Matches per Season')
fig_match_counts.add_hline(y=380, line_dash="dash", line_color="red", annotation_text="Standard Line (380 Matches)")
fig_match_counts.show()

Data loading and season labeling complete.

--- Season by Match Count ---
       season  match_count
0   2014-2015          380
1   2015-2016          380
2   2016-2017          380
3   2017-2018          380
4   2018-2019          380
5   2019-2020          380
6   2020-2021          380
7   2021-2022          380
8   2022-2023          380
9   2023-2024          380
10  2024-2025          380


### 1. EDA 결과 해석 (Interpretation of EDA Results)

- English: The dataset is clean and robust, containing 4,180 match records with 20 columns and no missing values. 

- Korean: 이 데이터셋은 4,180개의 경기 기록을 담고 있으며 결측치가 전혀 없는 매우 깨끗한 데이터입니다.

# 2단계: 탐색적 데이터 분석 (EDA - Exploratory Data Analysis)

In [2]:
# 2-1. 데이터 기본 정보 확인 (Initial Data Inspection)
print("--- Data Basic Information ---")
df.info()

print("\n\n--- Descriptive Statistics ---")
print(df.describe())

print("\n\n--- Missing Value Check ---")
print(df.isnull().sum())

# 2-2. 주요 변수 간 상관관계 분석 (Correlation Analysis of Key Variables)
corr_matrix = df[['home_goals', 'away_goals', 'home_xg', 'away_xg', 'home_shots', 'away_shots', 'home_sot', 'away_sot', 'home_deep', 'away_deep', 'home_ppda', 'away_ppda']].corr()

fig_corr = px.imshow(corr_matrix, text_auto=True, aspect="auto",
                     title="Correlation Heatmap of Key Variables")
fig_corr.show()

--- Data Basic Information ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4180 entries, 0 to 4179
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   match_id       4180 non-null   int64         
 1   date           4180 non-null   datetime64[ns]
 2   home_team      4180 non-null   object        
 3   away_team      4180 non-null   object        
 4   home_goals     4180 non-null   int64         
 5   away_goals     4180 non-null   int64         
 6   home_xg        4180 non-null   float64       
 7   away_xg        4180 non-null   float64       
 8   home_shots     4180 non-null   int64         
 9   away_shots     4180 non-null   int64         
 10  home_sot       4180 non-null   int64         
 11  away_sot       4180 non-null   int64         
 12  home_deep      4180 non-null   int64         
 13  away_deep      4180 non-null   int64         
 14  home_ppda      4180 non-null   float64   

### 2. Correlation Heatmap of Key Variables (주요 변수 간 상관관계 히트맵)

#### English
	•	The strongest relationships appear between expected goals (xG) and shots on target (SoT) for both home and away teams (≈0.66–0.67).
	•	This is intuitive: the more accurate shots a team produces, the higher its expected goals.
	•	Also, total shots and shots on target show a strong correlation (≈0.64–0.65), since SoT is a subset of shots.
	•	Finally, away xG and away goals correlate at 0.64, showing that xG is a meaningful predictor of actual scoring.

#### Korean (한국어)
	•	가장 강한 상관관계는 기대 득점(xG) 과 유효 슈팅(SoT) 사이에서 나타났습니다 (약 0.66–0.67).
	•	이는 직관적으로 타당합니다. 유효 슈팅이 많을수록 득점 기대값이 높아지기 때문입니다.
	•	또한 총 슈팅 수와 유효 슈팅 수 역시 강한 상관관계(0.64~0.65)를 보였습니다. 이는 유효 슈팅이 슈팅의 부분집합이기 때문입니다.
	•	마지막으로, 원정 xG와 원정 득점 간 상관관계가 0.64로 나타나, xG가 실제 득점을 설명하는 데 중요한 지표임을 확인할 수 있습니다. 

#  3단계: La Liga 전체 트렌드 분석 (League-wide Trend Analysis)

In [3]:
# 3-1. 시즌별 경기당 평균 득점 추이 (Seasonal Goals per Match Trend)
season_goals = df.groupby("season").apply(
    lambda x: (x["home_goals"] + x["away_goals"]).sum() / len(x)
).reset_index(name="goals_per_match")

fig_season_goals = px.line(
    season_goals,
    x="season",
    y="goals_per_match",
    markers=True,
    title="Seasonal Average Goals per Match",
    labels={"season": "Season", "goals_per_match": "Goals per Match"}
)
fig_season_goals.show()

# 3-2. 시즌별 홈/원정 승률 변화 (Seasonal Home/Away Win Rate Changes)
df["result"] = np.where(
    df["home_goals"] > df["away_goals"], "Home Win",
    np.where(df["home_goals"] < df["away_goals"], "Away Win", "Draw")
)
res_counts = df.groupby(["season", "result"]).size().unstack(fill_value=0)
res_rates = res_counts.div(res_counts.sum(axis=1), axis=0)

fig_win_rate = go.Figure()
fig_win_rate.add_bar(name='Home Win', x=res_rates.index, y=res_rates['Home Win'])
fig_win_rate.add_bar(name='Draw', x=res_rates.index, y=res_rates['Draw'])
fig_win_rate.add_bar(name='Away Win', x=res_rates.index, y=res_rates['Away Win'])

fig_win_rate.update_layout(
    barmode='stack',
    title="Changes in Home/Draw/Away Win Rates by Season",
    xaxis_title="Season",
    yaxis_title="Rate",
    yaxis_tickformat='.0%'
)
fig_win_rate.show()

/var/folders/yl/3cpbqsq96s70dm9lcf8p9c_00000gn/T/ipykernel_3532/3018619177.py:2: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



3. Seasonal Average Goals per Match (시즌별 평균 득점 추세)

- **English**: This line chart shows the average number of goals scored per match across La Liga seasons from 2014–15 to 2024–25.  
  The peak was in the 2016–17 season (around 2.93 goals per match), followed by a decline until 2019–20. Recently, goal averages have stabilized around 2.6.  
  This indicates cyclical patterns in offensive intensity across the league.  

- **한국어**: 이 선 그래프는 2014–15 시즌부터 2024–25 시즌까지 라리가 경기당 평균 득점 수를 보여줍니다.  
  2016–17 시즌에 약 2.9득점으로 정점을 찍은 뒤 2019–20 시즌까지 감소했으며, 최근에는 약 2.6득점 수준에서 안정세를 보입니다.  
  이는 라리가 전체 공격 강도가 일정한 주기적 변화를 겪고 있음을 의미합니다.  


---
 3. Home/Draw/Away Win Rates (홈/무/원정 승률 변화)
- **English**  
The stacked bar chart illustrates the seasonal distribution of home wins, draws, and away wins.  
Normally, home teams achieve the highest share of wins, reflecting the well-known *home advantage*.  

- However, in the **2020–2021 season**, the home win rate declined significantly, while away wins increased.  
This coincided with the COVID-19 pandemic, during which matches were played behind closed doors.  
The absence of fans diminished the psychological and environmental benefits usually enjoyed by home teams.  

---

- **Korean (한국어)**  
이 누적 막대 그래프는 시즌별 **홈팀 승리 / 무승부 / 원정팀 승리 비율**을 보여줍니다.  
일반적으로 홈팀의 승률이 가장 높으며, 이는 전통적인 *홈 어드밴티지*를 반영합니다.  

- 그러나 **2020–2021 시즌**에는 홈팀 승률이 크게 하락하고 원정팀 승률이 상승했습니다.  
이 시기는 코로나19로 인해 **무관중 경기**가 치러진 시즌으로, 관중 응원과 같은 홈 어드밴티지가 줄어든 결과로 해석할 수 있습니다.  

---

📌 **Portfolio Note**  
- The drop in home win rates in **2020–2021** aligns with the absence of fans during the COVID-19 pandemic.  
- Interpreting this change as a shift in *home advantage* makes the analysis more professional and research-oriented.  

# 4단계: Big 3 공격효율성 분석 (Big 3 Attack Efficiency Trend Analysis (GF – xGF per Match)
)

In [4]:
# 4-1. 데이터 재구성 (Restructure Data)
home_df = df[['season', 'home_team', 'home_goals', 'home_xg']].rename(columns={'home_team': 'team', 'home_goals': 'GF', 'home_xg': 'xGF'})
away_df = df[['season', 'away_team', 'away_goals', 'away_xg']].rename(columns={'away_team': 'team', 'away_goals': 'GF', 'away_xg': 'xGF'})
team_df = pd.concat([home_df, away_df], ignore_index=True)

# 4-2. 이름 표준화 함수 및 딕셔너리 정의 (Define Name Standardization)
big3_alias = {
    "Real Madrid": ["Real Madrid"],
    "Barcelona":   ["Barcelona", "FC Barcelona"],
    "Atletico Madrid": ["Atletico Madrid", "Atlético Madrid"]
}
def standardize_name(team_name):
    for standard_name, aliases in big3_alias.items():
        if team_name in aliases:
            return standard_name
    return team_name
team_df['team_std'] = team_df['team'].apply(standardize_name)

# 4-3. Big 3 팀 필터링 및 시즌별 집계 (Filter and Aggregate)
big3_teams = ["Real Madrid", "Barcelona", "Atletico Madrid"]
big3_df = team_df[team_df['team_std'].isin(big3_teams)]
season_team_agg = big3_df.groupby(['season', 'team_std']).agg(
    GF_total=('GF', 'sum'),
    xGF_total=('xGF', 'sum'),
    matches=('team_std', 'size')
).reset_index().rename(columns={'team_std': 'team'})

# 4-4. 경기당 지표 및 공격 효율성 계산 (Calculate Metrics)
season_team_agg['GF_per_match'] = season_team_agg['GF_total'] / season_team_agg['matches']
season_team_agg['xGF_per_match'] = season_team_agg['xGF_total'] / season_team_agg['matches']
season_team_agg['attack_efficiency'] = season_team_agg['GF_per_match'] - season_team_agg['xGF_per_match']

# 4-5. [수정됨] 공격 효율성 시각화 (REVISED: Visualize Attack Efficiency with Labels)
fig_efficiency = px.bar(
    season_team_agg,
    x="season",
    y="attack_efficiency",
    color="team",
    barmode='group',
    title="Big 3 Attack Efficiency Trend (GF - xGF per Match)",
    labels={
        "season": "Season",
        "attack_efficiency": "Attack Efficiency (GF - xGF per Match)",
        "team": "Team"
    },
    text_auto=True  # 막대에 텍스트를 자동으로 추가하는 기능
)

# 텍스트 라벨의 소수점 자릿수와 위치를 조정하여 가독성을 높입니다.
fig_efficiency.update_traces(texttemplate='%{y:.2f}', textposition='outside')
fig_efficiency.add_hline(y=0, line_width=2, line_dash="dash", line_color="black")
fig_efficiency.show()

## 4. Graph Interpretation (그래프 해석)

# **Definition**  
  - Attack Efficiency = GF_per_match − xGF_per_match  

# 🔵 Atletico Madrid
	-	Mostly positive values until late 2010s → slightly more goals than expected.
	-	2019-20 & 2024-25: Negative values → attack efficiency dropped.
	-	Overall stable, but recent decline in finishing quality observed.

# 🔴 Barcelona
	-	2014–2017: Large positive margin → Messi’s prime years, highly efficient finishing.
	-	2018 onward: Gradual decline, with 2022-23 below zero → inefficient finishing.
	-	Recently slight recovery, but no longer dominant as in the past.

# 🟢 Real Madrid
	-	2014–2017: Strong positive values → Ronaldo era, elite finishing efficiency.
	-	2018 onward: Gradual decline, turning negative in 2024-25.
	-	Reflects post-Ronaldo transition and reliance on new attacking structures.
===

# 🔵 Atletico Madrid
	•	대부분 시즌에서 +값(0 이상) → 기대득점 대비 실제 득점이 조금 더 많았음.
	•	하지만 2019-20, 2024-25 시즌에는 마이너스로 떨어짐 → 공격 효율이 기대치보다 낮았음을 의미.
	•	전반적으로 안정적인 득점 효율을 보여줬으나, 최근 시즌들에는 득점력 하락 조짐이 있음.

# 🔴 Barcelona
	•	2014-2017 시즌에는 GF-xGF가 크게 양수 → 메시 전성기 시절, 기대득점보다 훨씬 많은 득점을 기록.
	•	그러나 2018-19 이후 점차 줄어들어 2022-23 시즌에는 음수 → 기대득점보다 덜 득점하는 비효율적인 공격력.
	•	최근 다시 소폭 회복했으나, 과거만큼의 압도적 효율은 사라짐.

# 🟢 Real Madrid
	•	2014-2017 시즌 압도적 양수 → 호날두 시절, 기회보다 훨씬 많은 득점 (골 결정력 최강).
	•	2018-2019 이후 점차 하락, 최근 시즌(2024-25)에서는 오히려 음수 → 기대득점보다 덜 넣는 모습.
	•	이는 호날두 이탈 + 벤제마 이후 공격 전환 과정에서 나타난 변화로 해석 가능.

# 5단계 Defensive Analysis: Goals Conceded & PPDA

0) What & Why (영/한 요약)
-	•	PPDA = Passes allowed Per Defensive Action (상대에게 허용한 패스 수 ÷ 수비행동 수).
 → 낮을수록 강한 압박(더 공격적으로 수비).
-	•	GA (Goals Against) = 실점.
-	•	xGA (Expected Goals Against) = 실점이 예상된 값(상대가 만든 xG).
-	•	Defensive Efficiency (수비 효율성) = xGA_per_match − GA_per_match
 → 양수(+): 예상보다 실점을 적게 함(좋음), 음수(−): 예상보다 많이 실점(나쁨).

1) Import & Data Load (임포트/데이터 로드)

In [9]:
# 1) Import & Load
import pandas as pd
import numpy as np
import plotly.express as px

DF_PATH = "la_liga_2014_2025_all_matches_final.csv"  # 파일이 현재 폴더에 있어야 함
df = pd.read_csv(DF_PATH, parse_dates=["date"])
print(df.shape, df.columns.tolist()[:12])  # 빠른 확인

(4180, 19) ['match_id', 'date', 'home_team', 'away_team', 'home_goals', 'away_goals', 'home_xg', 'away_xg', 'home_shots', 'away_shots', 'home_sot', 'away_sot']


2) Season Normalization (시즌 표기 통일 + 순서 고정)

- EN: Normalize season labels to a standard form like 2016-17, then set a categorical order so x-axis doesn’t shuffle.
- KR: 시즌 표기를 2016-17 형식으로 통일하고, 카테고리 순서를 지정해 x축 순서를 고정합니다.

In [10]:
# 2) Season normalization
SEASON_ORDER = ["2014-15","2015-16","2016-17","2017-18","2018-19",
                "2019-20","2020-21","2021-22","2022-23","2023-24","2024-25"]

def normalize_season_label(s: str) -> str:
    if pd.isna(s):
        return s
    s = str(s).strip().replace("–","-").replace("—","-").replace("/", "-").replace(" ", "")
    mapping = {
        "2014-2015":"2014-15","2015-2016":"2015-16","2016-2017":"2016-17",
        "2017-2018":"2017-18","2018-2019":"2018-19","2019-2020":"2019-20",
        "2020-2021":"2020-21","2021-2022":"2021-22","2022-2023":"2022-23",
        "2023-2024":"2023-24","2024-2025":"2024-25"
    }
    return mapping.get(s, s)

# If 'season' is missing, map from 'date' using given boundaries
if "season" not in df.columns:
    SEASON_BOUNDARIES = {
        "2014-15": ("2014-08-23","2015-05-24"),
        "2015-16": ("2015-08-21","2016-05-15"),
        "2016-17": ("2016-08-19","2017-05-21"),
        "2017-18": ("2017-08-18","2018-05-20"),
        "2018-19": ("2018-08-17","2019-05-26"),
        "2019-20": ("2019-08-16","2020-07-19"),
        "2020-21": ("2020-09-12","2021-05-23"),
        "2021-22": ("2021-08-13","2022-05-22"),
        "2022-23": ("2022-08-12","2023-06-04"),
        "2023-24": ("2023-08-11","2024-05-26"),
        "2024-25": ("2024-08-15","2025-05-25"),
    }
    def season_from_date(d):
        d = pd.to_datetime(d).date()
        for sn,(s,e) in SEASON_BOUNDARIES.items():
            if pd.to_datetime(s).date() <= d <= pd.to_datetime(e).date():
                return sn
        return None
    df["season"] = df["date"].apply(season_from_date)

# Normalize & set order
df["season"] = df["season"].apply(normalize_season_label)
df["season"] = pd.Categorical(df["season"], categories=SEASON_ORDER, ordered=True)

print("NaN seasons:", int(df["season"].isna().sum()))

NaN seasons: 0


3) Build Team-Game Defensive Rows (팀-경기 수비 행 만들기)

- EN: For defense, the opponent’s goals/xG become our GA/xGA. PPDA comes from our team’s PPDA column.
- KR: 수비 관점에서는 상대 득점/xG가 우리의 실점(GA)/예상 실점(xGA) 이 됩니다. PPDA는 우리팀 PPDA를 사용합니다

In [11]:
# 3) Team-game defensive rows
home_def = df[["season","home_team","away_goals","away_xg","home_ppda"]].rename(
    columns={"home_team":"team","away_goals":"GA","away_xg":"xGA","home_ppda":"PPDA"}
)
away_def = df[["season","away_team","home_goals","home_xg","away_ppda"]].rename(
    columns={"away_team":"team","home_goals":"GA","home_xg":"xGA","away_ppda":"PPDA"}
)
team_def = pd.concat([home_def, away_def], ignore_index=True)
team_def[["GA","xGA","PPDA"]] = team_def[["GA","xGA","PPDA"]].astype(float)

# (optional) harmonize team names
def norm_team(s):
    if pd.isna(s): return s
    return (str(s).strip()
            .replace("Atlético Madrid","Atletico Madrid")
            .replace("FC Barcelona","Barcelona")
            .replace("Real Madrid CF","Real Madrid"))
team_def["team"] = team_def["team"].apply(norm_team)

team_def.head()

,season,team,GA,xGA,PPDA
0,2014-15,Malaga,0.0,1.14,12.07
1,2014-15,Sevilla,1.0,1.75,9.42
2,2014-15,Granada,1.0,0.38,4.78
3,2014-15,Almeria,1.0,0.40,6.67
4,2014-15,Eibar,0.0,0.98,8.79


4) Aggregate per Season-Team & Compute Defensive Efficiency

    (시즌·팀 집계 + 수비 효율성 계산)

-    EN: Aggregate totals/means, convert to per-match, and define
-    def_efficiency = xGA_per_match − GA_per_match (positive = good).
-    KR: 합계를 시즌·팀 단위로 집계 후 경기당 지표로 환산,
-    def_efficiency = xGA/m − GA/m 으로 정의합니다 (양수=좋음).

In [22]:
# 4) Aggregate & compute per-match + efficiency
grp = (team_def
       .dropna(subset=["season","team"])            # 시즌/팀 결측 제거
       .groupby(["season","team"], observed=True)   # 관측된 조합만 사용
)

season_team_def = grp.agg(
    GA_total=("GA","sum"),
    xGA_total=("xGA","sum"),
    matches=("team","size"),
    PPDA_avg=("PPDA","mean")
).reset_index()   # <- as_index=True 상태에서 마지막에 인덱스 복구

# 이후 계산은 그대로 유지
season_team_def["GA_per_match"]  = season_team_def["GA_total"]  / season_team_def["matches"]
season_team_def["xGA_per_match"] = season_team_def["xGA_total"] / season_team_def["matches"]
season_team_def["def_efficiency"] = season_team_def["xGA_per_match"] - season_team_def["GA_per_match"]

# 보기 좋게 2dp
num_cols = ["GA_total","xGA_total","matches","PPDA_avg","GA_per_match","xGA_per_match","def_efficiency"]
season_team_def[num_cols] = season_team_def[num_cols].round(2)

# Defensive Efficiency (positive is good)
season_team_def["def_efficiency"] = season_team_def["xGA_per_match"] - season_team_def["GA_per_match"]

# 2dp rounding for neat output (tables/hover)
num_cols = ["GA_total","xGA_total","matches","PPDA_avg","GA_per_match","xGA_per_match","def_efficiency"]
season_team_def[num_cols] = season_team_def[num_cols].round(2)

# Order by season then team
season_team_def["season"] = pd.Categorical(season_team_def["season"], categories=SEASON_ORDER, ordered=True)
season_team_def = season_team_def.sort_values(["season","team"]).reset_index(drop=True)

season_team_def.head(20)

,season,team,GA_total,xGA_total,matches,PPDA_avg,GA_per_match,xGA_per_match,def_efficiency
0,2014-15,Almeria,64.0,58.86,38,8.76,1.68,1.55,-0.13
1,2014-15,Athletic Club,41.0,44.09,38,7.46,1.08,1.16,0.08
2,2014-15,Atletico Madrid,29.0,29.08,38,8.98,0.76,0.77,0.01
3,2014-15,Barcelona,21.0,28.47,38,5.68,0.55,0.75,0.20
4,2014-15,Celta Vigo,44.0,51.73,38,6.06,1.16,1.36,0.20
5,2014-15,Cordoba,68.0,57.33,38,11.61,1.79,1.51,-0.28
6,2014-15,Deportivo La Coruna,60.0,50.99,38,9.87,1.58,1.34,-0.24
7,2014-15,Eibar,55.0,53.70,38,9.64,1.45,1.41,-0.04
8,2014-15,Elche,62.0,58.05,38,9.52,1.63,1.53,-0.10
9,2014-15,Espanyol,51.0,48.29,38,9.38,1.34,1.27,-0.07


5) Visualizations (시각화: 소수점 둘째자리)

-    5-a) League-wide: PPDA vs GA per match (리그 전체 산점도)

-    EN: If the cloud slopes up-right, weaker pressing (higher PPDA) links to more goals conceded.
-    KR: 오른쪽 위로 기울면 압박이 약할수록(PPDA 높음) 실점이 많아짐을 시사.

In [14]:
# 5-a) League-wide scatter
fig_scatter = px.scatter(
    season_team_def, x="PPDA_avg", y="GA_per_match", color="season",
    labels={"PPDA_avg":"PPDA (lower = stronger press)", "GA_per_match":"GA per Match"},
    title="League-wide: PPDA vs Goals Conceded per Match",
    hover_data=["team","season"]
)
fig_scatter.update_traces(hovertemplate="Team: %{customdata[0]}<br>Season: %{customdata[1]}<br>PPDA: %{x:.2f}<br>GA/m: %{y:.2f}")
fig_scatter.update_xaxes(tickformat=".2f")
fig_scatter.update_yaxes(tickformat=".2f")
fig_scatter.show()

5-b) Big 3 Defensive Efficiency (빅3 수비 효율성 막대)

-    EN: Positive bars mean “better than expected” defense.
-    KR: 막대가 양수면 “예상보다 실점이 적어 수비가 좋았음”을 뜻합니다.

In [18]:
# 빅3 필터 + 그래프(hover fix 포함)
big3 = ["Real Madrid","Barcelona","Atletico Madrid"]
big3_def = season_team_def[season_team_def["team"].isin(big3)].copy()
big3_def["season"] = pd.Categorical(big3_def["season"], categories=SEASON_ORDER, ordered=True)

lab = big3_def.copy()
lab["ae_txt"] = lab["def_efficiency"].apply(lambda v: f"{v:.2f}" if pd.notna(v) and abs(v) >= 0.03 else "")
pad = 0.02
rng = float(lab["def_efficiency"].abs().max()) if not lab.empty else 0.1
ylim = max(0.10, round(rng + pad, 2))

import plotly.express as px
fig_defeff = px.bar(
    lab, x="season", y="def_efficiency", color="team",
    barmode="group", category_orders={"season": SEASON_ORDER},
    labels={"season":"Season / 시즌","def_efficiency":"Defensive Efficiency (xGA/m − GA/m)"},
    title="Big 3 Defensive Efficiency (xGA/m − GA/m)",
    text="ae_txt",
    custom_data=["team"]   # ← hover용 팀명 전달
)
fig_defeff.update_traces(
    textposition="outside",
    marker_line_width=0.5, marker_line_color="black",
    hovertemplate="Season %{x}<br>Team: %{customdata[0]}<br>Def Eff: %{y:.2f}<extra></extra>"
)
fig_defeff.update_yaxes(tickformat=".2f", range=[-ylim, ylim])
fig_defeff.add_hline(y=0, line_dash="dash", line_width=2, line_color="black")
fig_defeff.add_hrect(y0=-0.03, y1=0.03, fillcolor="lightgray", opacity=0.18, line_width=0, layer="below")
fig_defeff.show()

5-c) Big 3 PPDA Trend (빅3 PPDA 추세)

    - EN: Lower PPDA → stronger pressing. See if drops in PPDA align with lower GA/m.
    - KR: PPDA 하락 = 압박 강화. PPDA 하락이 GA/m 하락과 함께 움직이는지 확인하세요.

In [19]:
fig_ppda = px.line(
    big3_def, x="season", y="PPDA_avg", color="team", markers=True,
    category_orders={"season": SEASON_ORDER},
    title="Big 3 PPDA Trend (lower = stronger pressing)"
)
fig_ppda.update_traces(hovertemplate="Season %{x}<br>%{legendgroup}: PPDA %{y:.2f}")
fig_ppda.update_yaxes(tickformat=".2f")
fig_ppda.show()

5-d) Big 3 GA per Match Trend (빅3 경기당 실점 추세)

    - EN: Downward trend suggests improved defensive outcomes.
    - KR: 하락 추세면 수비 결과(실점)가 개선되는 방향입니다.

In [20]:
fig_ga = px.line(
    big3_def, x="season", y="GA_per_match", color="team", markers=True,
    category_orders={"season": SEASON_ORDER},
    title="Big 3 Goals Conceded per Match Trend"
)
fig_ga.update_traces(hovertemplate="Season %{x}<br>%{legendgroup}: GA/m %{y:.2f}")
fig_ga.update_yaxes(tickformat=".2f")
fig_ga.add_hline(y=0, line_dash="dash")
fig_ga.show()

- EN

    Across 2014–2025, Real Madrid deliver the steadiest defensive overperformance (positive DefEff) despite not always pressing aggressively (higher PPDA), suggesting effectiveness driven by box protection, shot-quality suppression, and goalkeeping. Barcelona’s best defensive seasons coincide with lower PPDA, indicating a clearer payoff from structured high pressing. Atlético’s 2021–22 stands out as a defensive underperformance (negative DefEff), with subsequent recovery; longer-term PPDA drift upward hints at a shift away from intense pressing.

- KR

    2014–2025 전기간을 보면, 레알은 압박( PPDA )이 항상 강하지 않아도 **지속적으로 기대 대비 실점을 줄이는 효율(양의 DefEff)**을 보여 박스 보호/슈팅 퀄리티 관리/GK 퍼포먼스의 기여가 컸음을 시사합니다. 바르셀로나는 **낮은 PPDA(강한 압박)**일수록 GA/m 하락과 DefEff 개선이 동행해 구조화된 전방 압박의 효과가 비교적 명확합니다. 아틀레티코는 2021–22에 음의 DefEff로 기대 이하의 수비 성과가 두드러졌고 이후 회복했으며, 장기적으로 **PPDA 상승(압박 약화)**이 관측됩니다.

In [21]:
from pathlib import Path
import plotly.io as pio

OUT = Path("outputs"); OUT.mkdir(exist_ok=True)

# 2dp summary for Big3
export_cols = ["season","team","GA_per_match","xGA_per_match","def_efficiency","PPDA_avg","matches"]
tmp = big3_def[export_cols].copy().sort_values(["team","season"])
tmp.to_csv(OUT/"big3_defense_summary_2dp.csv", index=False)

# HTML charts
pio.write_html(fig_scatter, file=OUT/"league_PPDA_vs_GA.html", include_plotlyjs="cdn")
pio.write_html(fig_defeff, file=OUT/"big3_def_efficiency.html", include_plotlyjs="cdn")
pio.write_html(fig_ppda,   file=OUT/"big3_ppda_trend.html", include_plotlyjs="cdn")
pio.write_html(fig_ga,     file=OUT/"big3_ga_trend.html", include_plotlyjs="cdn")